In [2]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate,ChatPromptTemplate


chat = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1, streaming=True)





In [4]:
chef_template = ChatPromptTemplate.from_messages([
    ("system","You are a world class chef. You are given a list of ingredients easy to find in a kitchen and you need to create a recipe for a dish easy to make."),
    ("human","I want to cook {cousine}food."),
])

chef_chain = chef_template | chat



In [7]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetarian chef. specialized in traditional recepis vegeterian. you find alternatives for non-vegetarian ingredients. you don't radically modify the recipe. if there is alternative for a food just say you don't know how to replace it"),
    ("human","{recipe}"),
])

veg_chef_chain = veg_chef_prompt | chat


final_chain = {"recipe":chef_chain} | veg_chef_chain


final_chain.invoke({"cousine":"Indian"})








AIMessage(content='This is a wonderful vegetarian dish! Since all the ingredients are plant-based, you can prepare this Chana Masala as is. Just ensure that the canned chickpeas are vegetarian (most are), and use vegetable oil. If you want to make it more hearty, you can add vegetables like spinach or potatoes, but the recipe as provided is already fully vegetarian and delicious. Enjoy your cooking!')

In [37]:

from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        items = text.strip().split(",")
        return list(map(str.strip, items))


comma_parser = CommaOutputParser()
comma_parser.parse("Hello,how ,are,you,!")
template = ChatPromptTemplate.from_messages([
   ("system", "You are a list generating machine. Everything you area aksed will be answerd with a comma separated list of max {max_items} in lowercase. Do not reply with antyhing else"),
    
    ("human", "{question}"),
])

prompt = template.format_messages(max_items=10, question="What are the colors?")
response = chat.predict_messages(prompt)

comma_parser = CommaOutputParser()
comma_parser.parse(response.content)

['red',
 'blue',
 'yellow',
 'green',
 'orange',
 'purple',
 'black',
 'white',
 'gray',
 'pink']

In [38]:
chain = template | chat | comma_parser

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['bulbasaur', 'ivysaur', 'venusaur', 'charmander', 'charmeleon']

In [ ]:


chat = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)
template = PromptTemplate.from_template("What is the distance between {country1} and {country2}?")
prompt = template.format(country1="Mexico", country2="Thailand")


template = ChatPromptTemplate.from_messages([
   ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "안녕 내 이름은 {name}이야!"),
    ("human", "Hello, What is your name? And what is the distance between {country1} and {country2}? "),
])

prompt = template.format_messages(language="japanese", name="Socrates", country1="Mexico", country2="Thailand")

chat.predict_messages(prompt)

AIMessage(content='こんにちは、私の名前はSocraです。メキシコとタイの距離はおよそ13,000キロメートルです。')